In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Attention

In [2]:
?Attention

Init signature: Attention(use_scale=False, **kwargs)
Docstring:     
Dot-product attention layer, a.k.a. Luong-style attention.

Inputs are `query` tensor of shape `[batch_size, Tq, dim]`, `value` tensor of
shape `[batch_size, Tv, dim]` and `key` tensor of shape
`[batch_size, Tv, dim]`. The calculation follows the steps:

1. Calculate scores with shape `[batch_size, Tq, Tv]` as a `query`-`key` dot
   product: `scores = tf.matmul(query, key, transpose_b=True)`.
2. Use scores to calculate a distribution with shape
   `[batch_size, Tq, Tv]`: `distribution = tf.nn.softmax(scores)`.
3. Use `distribution` to create a linear combination of `value` with
   shape `[batch_size, Tq, dim]`:
   `return tf.matmul(distribution, value)`.

Args:
  use_scale: If `True`, will create a scalar variable to scale the attention
    scores.
  causal: Boolean. Set to `True` for decoder self-attention. Adds a mask such
    that position `i` cannot attend to positions `j > i`. This prevents the
    flow of inform

### Cross-Attention in Keras

In [4]:
a = Attention()

In [28]:
v = tf.constant([
    [
        [1, 2],
        [3, 4],
    ]
], dtype=tf.float32)
v.shape

TensorShape([1, 2, 2])

In [53]:
k = tf.constant([
    [
        [1, 0],
        [0, 1],
    ]
], dtype=tf.float32)
k.shape

TensorShape([1, 2, 2])

In [54]:
q = tf.constant([
    [
        [1, 0],
        [0, 1],
        [1, 1],
        [0, 0]
    ]
], dtype=tf.float32)
q.shape

TensorShape([1, 4, 2])

In [55]:
out = a([q, v, k])
out.shape

TensorShape([1, 4, 2])

In [56]:
out[0].numpy()

array([[1.5378828, 2.5378828],
       [2.462117 , 3.462117 ],
       [2.       , 3.       ],
       [2.       , 3.       ]], dtype=float32)

### Cross-Attention in Tensorflow

In [70]:
similar_logit = [qm @ tf.transpose(km) for qm, km in zip(q, k)]
similar_weight = tf.math.softmax(similar_logit)
out = similar_weight @ v
out.numpy()

array([[[1.5378828, 2.5378828],
        [2.462117 , 3.462117 ],
        [2.       , 3.       ],
        [2.       , 3.       ]]], dtype=float32)

### Self-Attention in Keras

In [75]:
a([k, k, k]).numpy()

array([[[0.73105854, 0.26894143],
        [0.26894143, 0.73105854]]], dtype=float32)

### Self-Attention in Tensorflow

In [76]:
similar_logit = [km @ tf.transpose(km) for km, km in zip(k, k)]
similar_weight = tf.math.softmax(similar_logit)
out = similar_weight @ k
out.numpy()

array([[[0.73105854, 0.26894143],
        [0.26894143, 0.73105854]]], dtype=float32)

### Multi-head Attention

In [80]:
from tensorflow.keras.layers import MultiHeadAttention
?MultiHeadAttention

Init signature:
MultiHeadAttention(
    num_heads,
    key_dim,
    value_dim=None,
    dropout=0.0,
    use_bias=True,
    output_shape=None,
    attention_axes=None,
    kernel_initializer='glorot_uniform',
    bias_initializer='zeros',
    kernel_regularizer=None,
    bias_regularizer=None,
    activity_regularizer=None,
    kernel_constraint=None,
    bias_constraint=None,
    **kwargs,
)
Docstring:     
MultiHeadAttention layer.

This is an implementation of multi-headed attention as described in the paper
"Attention is all you Need" (Vaswani et al., 2017).
If `query`, `key,` `value` are the same, then
this is self-attention. Each timestep in `query` attends to the
corresponding sequence in `key`, and returns a fixed-width vector.

This layer first projects `query`, `key` and `value`. These are
(effectively) a list of tensors of length `num_attention_heads`, where the
corresponding shapes are `(batch_size, <query dimensions>, key_dim)`,
`(batch_size, <key/value dimensions>, key_di